In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/churn.csv', sep= ',')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0,0,1,DIAMOND,300
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,0,5,PLATINUM,771
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1,1,3,SILVER,564
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,1,2,GOLD,339


In [3]:
df.shape

(10000, 18)

In [4]:
#Cálculo da frequência percentual do churn, sendo 0 os que ficaram e 1 os que sairam
#A função "normalize = True" retorna a proporção e ao final multiplicamos o valor por 100 para receber a porcentagem
churn_rate = df['Exited'].value_counts(normalize=True)*100
churn_rate

Exited
0    79.62
1    20.38
Name: proportion, dtype: float64

In [5]:
import plotly.express as px


churn_data = pd.DataFrame({'Status':['Permaneceu', 'Saiu'], 'Porcentagem': churn_rate.values})

fig = px.bar(churn_data, x='Status', y='Porcentagem',
             title='Taxa de Churn dos Clientes',
             labels={'Status' : 'Status,', 'Porcentagem' : 'Porcentagem (%)'},
             color='Status',
             color_discrete_map={'Permaneceu': 'green', 'Saiu' : 'red'})

fig.update_layout(yaxis_range=[0, 100])
fig.show()

In [6]:
#Seleciona a coluna 'Complain' e compara com a 'Exited', sendo 0 os que permaneceram e 1 os que sairam
#Unstack desempilha a coluna para uma melhor visualização em tabela
churn_by_complain = df.groupby('Complain')['Exited'].value_counts(normalize= 'True').unstack().round(4) * 100
churn_by_complain
#Pode-se analizar que a reclamação possui uma grande influência sobre o cancelamento do plano, ou seja, em algum momento do diálogo
#o cliente se sente insatisfeito com a resolução

Exited,0,1
Complain,,
0,99.95,0.05
1,0.49,99.51


In [7]:
import plotly.graph_objects as go

fig = go.Figure(data=[
                go.Bar(name= 'Permaneceu', x=churn_by_complain.index, y=churn_by_complain[0], marker_color= 'green'),
                go.Bar(name= 'Saiu', x=churn_by_complain.index, y=churn_by_complain[1], marker_color= 'red')
                ])

fig.update_layout(barmode= 'group',
                  title= 'Taxa de Churn com Reclamações',
                  xaxis_title= 'Reclamações (0 = Permaneceu, 1 = Saiu)',
                  yaxis_title= 'Porcentagem (%)',
                  xaxis = dict(tickmode = 'array', tickvals = [0, 1], ticktext = ['Sem reclamação', 'Com Reclamação']),
                  yaxis_range = [0, 100])

fig.show()

In [8]:
fig = px.histogram(df, x='Age',
                   title= 'Distribuição de Idade dos Clientes',
                   labels= {'Age', 'Idade'})
fig.show()

In [22]:
import plotly.express as px

# Criando o histograma com o mapeamento de cores
fig = px.histogram(df, x='Age', color='Exited', 
                   barmode='group', 
                   title='Distribuição de Idade por Churn',
                   # Mapeando 0 para azul e 1 para vermelho
                   color_discrete_map={0: 'blue', 1: 'red'})

# Opcional: Melhorar a legenda para ficar mais claro
fig.update_layout(legend_title_text='Status')
newnames = {'0':'Permaneceu', '1': 'Saiu'}
fig.for_each_trace(lambda t: t.update(name = newnames.get(t.name, t.name)))

fig.show()

In [23]:
# Filtrando e contando os tipos de cartão
df_filtrado = df.query('39 <= Age <= 53')
contagem_cartoes = df_filtrado['Card Type'].value_counts()
print(contagem_cartoes)

Card Type
DIAMOND     883
PLATINUM    872
SILVER      869
GOLD        846
Name: count, dtype: int64


In [ ]:
import plotly.express as px

# Filtrando os dados
df_range = df.query('39 <= Age <= 53')

# Criando o gráfico
fig = px.histogram(df_range, 
                   x='Card Type', 
                   color='Exited', # Adicionado para manter a lógica de churn
                   barmode='group',
                   title='Distribuição de Tipos de Cartão (Idades 39-53)',
                   labels={'Card Type': 'Tipo de Cartão', 'count': 'Quantidade de Clientes'},
                   color_discrete_map={0: 'blue', 1: 'red'}) # Mantendo o padrão de cores anterior

fig.show()

#Os gráficos seguintes que não há grande relação entre o tipo de cartão demonstrou que não há grande relação entre
#o tipo do cartão e a taxa de churn

In [25]:
# Criando a tabela de proporção
churn_by_card = df.groupby('Card Type')['Exited'].value_counts(normalize=True).unstack() * 100

# Arredondando para facilitar a leitura
churn_by_card = churn_by_card.round(2)
print(churn_by_card)

Exited         0      1
Card Type              
DIAMOND    78.22  21.78
GOLD       80.74  19.26
PLATINUM   79.64  20.36
SILVER     79.89  20.11


In [ ]:
import plotly.express as px

# O histograma com o argumento histfunc='avg' ou usando o DataFrame agrupado
# A forma mais simples de ver a relação direta é:
fig = px.histogram(df, 
                   x='Card Type', 
                   color='Exited', 
                   barmode='group',
                   histnorm='percent', # Mostra a proporção dentro de cada categoria
                   title='Relação entre Tipo de Cartão e Churn',
                   labels={'Card Type': 'Tipo de Cartão', 'Exited': 'Status (0=Ficou, 1=Saiu)'},
                   color_discrete_map={0: 'blue', 1: 'red'})

fig.update_layout(yaxis_title="Porcentagem (%)")
fig.show()

#Até este ponto é demonstrado a relação entre o tipo do cartão e o churn